## Import packages

In [1]:
## import self defined functions
from sys import path
import os
home = os.path.expanduser("~")
# insert at 1, 0 is the script path (or '' in REPL)
path.insert(1, '/tigress/cw55/local/python_lib')
from cg_funcs import global_mean_xarray, error_info, common_list

In [2]:
import numpy as np
import copy
import subprocess
import xarray as xr
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as LiRe
from sklearn.metrics import r2_score

In [3]:
from scipy import stats
from scipy.optimize import fsolve
def r2_for_give_pvalue(p_value,N_sample):
    '''
    return the r2 for give p_value and sample size
    assume BLUE OLS
    '''
    def func(r2):
#     def func(t_v):
        t_v = np.sqrt(r2*(N_sample-2)/(1-r2))
        p_value_r2 = stats.t.sf(abs(t_v), df=N_sample-2)*2
        return p_value_r2 - p_value
    return fsolve(func, 0.001)

# load model data

In [4]:
%%time

dvc_info = '2022ECSTCR'    # data version control
time_length_check = { "piControl": 200, "historical": 165, "hist-GHG": 165, "hist-aer": 165,
                      "abrupt-4xCO2": 150,  "1pctCO2": 150, "amip-4xCO2": 35, "amip-p4K": 35  }

experiments= ["piControl","abrupt-4xCO2","1pctCO2"]
# experiments= ["piControl"]

res_hori = '5x5'
var_list = "tas rlut rsdt rsut"
ds_exp_var = {}
for expri in experiments:
    tle = time_length_check[expri]
    ds_exp_var[expri] = {}
    for var in var_list.split():
        if expri == "1pctCO2" and var != 'tas': continue
        dirpath = f"$CMIP6post/CMIP6_post_regrid/{expri}/*/{var}/{var}.mon.0001-{tle:04d}.nc.r1i1p1*.{res_hori}.{dvc_info}"
        tmp = subprocess.run(["source ~/.bash_env;  ls -f "+dirpath], shell=True, capture_output=True)
        error_info(tmp.stderr.decode("utf-8"))
        filelist = tmp.stdout.decode("utf-8").split()
        with  xr.open_mfdataset( filelist,
                                 parallel=True,
                                 combine='by_coords',
                                 coords='minimal',
                                 compat='override',
                                 use_cftime=True) as data_tmp:
            ds_exp_var[expri][var] = data_tmp

CPU times: user 8.29 s, sys: 2.85 s, total: 11.1 s
Wall time: 8.86 s


In [5]:
# rename piControl ensemble for HadGEM3-GC31-xx
# f1 -> f3
pi_model = []
expri = 'piControl'
for var in var_list.split():
    pi_model = copy.deepcopy(ds_exp_var[expri][var].model.values)
    mi = []
    for i, m in enumerate(pi_model):
        if 'HadGEM3-GC31-' in m:
#             print(m)
            mi.append(i)
            m_n, m_e = m.split()
    for i in mi:
        m_n, m_e = pi_model[i].split()
        pi_model[i] = m_n+' '+m_e[:-2]+'f3'
#     print(pi_model[mi])
#     print(pi_model)
    ds_exp_var[expri][var].coords['model'] = pi_model
#     break

In [6]:
## find common model
model_lists = []

for expri in experiments:
    for var in var_list.split():
        if expri == "1pctCO2" and var != 'tas': continue
        model_lists.append(list(ds_exp_var[expri][var].model.values))

In [7]:
%%time
## find common model
model_lists = []

for expri in experiments:
    for var in var_list.split():
        if expri == "1pctCO2" and var != 'tas': continue
        model_lists.append(list(ds_exp_var[expri][var].model.values))
    
most_common_model = common_list(model_lists)
model_num = len(most_common_model)
print(most_common_model)

for expri in experiments:
    for var in var_list.split():
        if expri == "1pctCO2" and var != 'tas': continue
        ds_exp_var[expri][var] = ds_exp_var[expri][var].sel(model = most_common_model).load()
    

Com_len: 47 | Max_len: 57 | Min_len: 52
['ACCESS-CM2 r1i1p1f1', 'ACCESS-ESM1-5 r1i1p1f1', 'AWI-CM-1-1-MR r1i1p1f1', 'BCC-CSM2-MR r1i1p1f1', 'BCC-ESM1 r1i1p1f1', 'CAMS-CSM1-0 r1i1p1f1', 'CAS-ESM2-0 r1i1p1f1', 'CESM2 r1i1p1f1', 'CESM2-FV2 r1i1p1f1', 'CESM2-WACCM r1i1p1f1', 'CESM2-WACCM-FV2 r1i1p1f1', 'CIESM r1i1p1f1', 'CMCC-CM2-SR5 r1i1p1f1', 'CMCC-ESM2 r1i1p1f1', 'CNRM-CM6-1 r1i1p1f2', 'CNRM-CM6-1-HR r1i1p1f2', 'CNRM-ESM2-1 r1i1p1f2', 'CanESM5 r1i1p1f1', 'E3SM-1-0 r1i1p1f1', 'EC-Earth3-AerChem r1i1p1f1', 'EC-Earth3-CC r1i1p1f1', 'EC-Earth3-Veg r1i1p1f1', 'FGOALS-f3-L r1i1p1f1', 'FGOALS-g3 r1i1p1f1', 'FIO-ESM-2-0 r1i1p1f1', 'GFDL-CM4 r1i1p1f1', 'GFDL-ESM4 r1i1p1f1', 'GISS-E2-1-G r1i1p1f1', 'GISS-E2-1-H r1i1p1f1', 'HadGEM3-GC31-LL r1i1p1f3', 'HadGEM3-GC31-MM r1i1p1f3', 'INM-CM4-8 r1i1p1f1', 'INM-CM5-0 r1i1p1f1', 'IPSL-CM6A-LR r1i1p1f1', 'KACE-1-0-G r1i1p1f1', 'MIROC-ES2L r1i1p1f2', 'MIROC6 r1i1p1f1', 'MPI-ESM-1-2-HAM r1i1p1f1', 'MPI-ESM1-2-HR r1i1p1f1', 'MPI-ESM1-2-LR r1i1p1f1', 'MRI-ESM2

## Detrend on every model, month, grid (lat, lon) when the data has significant yearly trend. 

In [8]:
r2_bar = r2_for_give_pvalue(0.05, 200)
        
ds_exp_var_copy = copy.deepcopy(ds_exp_var)
# ######### test detrend on piControl data ############
# for var in var_list.split():
# #     for i, m in enumerate(model_hist):
#     for m in set(model_piCo):
#         print('>',end='')
#         data_pi = ds_exp_var['piControl'][var][var].sel(model=m)
#         data_pi = data_pi.values.reshape((200,-1)) # year, [month, lat, lon]
#         X   = np.arange(200)[:,None]
#         Y   = data_pi
#         reg = LiRe().fit(X, Y)
#         r2 = r2_score(Y, reg.predict(X), multioutput='raw_values')
#         data_trend = np.where(r2>r2_bar, X@reg.coef_.T, 0) # only detrend significant regions
# #         data_trend =  X@reg.coef_.T #  detrend all regions
#         data_detrend = data_pi-data_trend
#         ds_exp_var_copy['piControl'][var][var].sel(model=m)[:] = data_detrend.reshape((200*12,36,72))
# logging.info('piControl Detrend finished')
# ## GM
# model_lager_trend = []
# expri = 'piControl'
# var = 'tas'
# for m in set(model_piCo):
#     data = ds_exp_var_copy[expri][var][var].sel(model=m).groupby('time.year').mean('time').compute()
#     data = global_mean_xarray(data)
#     ptrend, pvalue = reg_pi_trend(data)
#     if pvalue < 0.01 and ptrend > 0.1:
#         print(m, ptrend, pvalue)
#         model_lager_trend.append(m)
# print(f'models with significant (p<0.01) and large (dT > 0.1 /150 years) trend: \n {model_lager_trend}')
# ## NH-SH
# expri = 'piControl'
# var = 'tas'
# for m in set(model_piCo):
#     data = ds_exp_var_copy[expri][var][var].sel(model=m).groupby('time.year').mean('time').compute()
#     data = global_mean_xarray(data.sel(lat=slice(  0,90))) \
#           - global_mean_xarray(data.sel(lat=slice(-90, 0)))
#     ptrend, pvalue = reg_pi_trend(data)
#     if pvalue < 0.01 and ptrend > 0.1:
#         print(m, ptrend, pvalue)
#         model_lager_trend.append(m)
# print(f'models with significant (p<0.01) and large (dT > 0.1 /150 years) trend: \n {model_lager_trend}')

for expri in ["abrupt-4xCO2","1pctCO2"]:
    print(f'{expri}')
    tle = time_length_check[expri]
    for var in var_list.split():
        if expri == "1pctCO2" and var != 'tas': continue
        print(f'{var}')
        for i, m in enumerate(most_common_model): 
            print('>',end='')
            data_pi = ds_exp_var['piControl'][var][var].sel(model=most_common_model[i])
            data_pi = data_pi.values.reshape((200,-1)) # year, [month, lat, lon]
            X   = np.arange(200)[:,None]
            Y   = data_pi
            reg = LiRe().fit(X, Y)

            r2 = r2_score(Y, reg.predict(X), multioutput='raw_values')
            X   = np.arange(tle)[:,None]
            data_trend = np.where(r2>r2_bar, X@reg.coef_.T, 0) # only detrend significant regions # L1
#             data_trend =  X@reg.coef_.T #  detrend all regions # L2
            data_hi = ds_exp_var[expri][var][var].sel(model=m)
            data_hi = data_hi.values.reshape((tle,-1)) # year, [month, lat, lon]
            data_detrend = data_hi-data_trend
            ds_exp_var_copy[expri][var][var].sel(model=m)[:] = data_detrend.reshape((tle*12,36,72))
        print()
    print(f'{expri} Detrend finished')


abrupt-4xCO2
tas
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
rlut
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
rsdt
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
rsut
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
abrupt-4xCO2 Detrend finished
1pctCO2
tas
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
1pctCO2 Detrend finished


In [9]:
%%time
var = 'tas'
dtas_gm_4xCO2  = global_mean_xarray(ds_exp_var_copy['abrupt-4xCO2'][var][var].groupby('time.year').mean('time'))\
                -global_mean_xarray(ds_exp_var_copy['piControl'][var][var].mean(dim='time'))
dtas_gm_4xCO2 = dtas_gm_4xCO2.load()
var = 'rlut rsdt rsut'
dR_gm_4xCO2 = {}
for _var in var.split():
    dR_gm_4xCO2[_var] = global_mean_xarray(ds_exp_var_copy['abrupt-4xCO2'][_var][_var].groupby('time.year').mean('time'))\
                       -global_mean_xarray(ds_exp_var_copy['piControl'][_var][_var].mean(dim='time'))
dR_gm_4xCO2 = dR_gm_4xCO2['rsdt'] - dR_gm_4xCO2['rsut'] - dR_gm_4xCO2['rlut']
dR_gm_4xCO2 = dR_gm_4xCO2.load()

CPU times: user 6.2 s, sys: 2.04 s, total: 8.23 s
Wall time: 8.25 s


In [10]:
%%time
## global mean
var = 'tas'
dtas_gm_1pct  = global_mean_xarray(ds_exp_var_copy['1pctCO2'][var][var].groupby('time.year').mean('time'))\
                -global_mean_xarray(ds_exp_var_copy['piControl'][var][var].mean(dim='time'))
dtas_gm_1pct = dtas_gm_1pct.load()

CPU times: user 1.82 s, sys: 391 ms, total: 2.21 s
Wall time: 2.22 s


# Save data (global mean values)

In [11]:
%%time
ds_data = xr.Dataset(
    data_vars=dict(
        dtas_gm_1pctCO2      = (["model", "year"], dtas_gm_1pct.values),
        dtas_gm_abrupt_4xCO2 = (["model", "year"], dtas_gm_4xCO2.values),
        dR_gm_abrupt_4xCO2   = (["model", "year"], dR_gm_4xCO2.values),
    ),
    coords=dict(
        model=(["model"], most_common_model),
        year=(["year"], np.arange(150)),
    ),
    attrs=dict(description="CMIP6, model surface air temperature change in 1pctCO2 and abrupt_4xCO2 experiments."),
)

CPU times: user 771 µs, sys: 0 ns, total: 771 µs
Wall time: 748 µs


In [12]:
file_name = 'CMIP6_dtas_dR_data.detrend_L1.nc'
ds_data.to_netcdf(file_name)

## 